In [1]:
# Group Members: Ariel Liu, Yuqin Wang, Zihao Wang, Huanlun Li
# Starting Date: 11.21.18

# Our Project is mainly targeting on Twitter Crawling, allowing users to give in user input for specific tweet
# searches. We will make a word map according to the frequencies of words the user chooses to post in his/her recent
# tweets. We will input President Donald Trump's user ID as a demo in the end of our project.

In [ ]:

# we use utf-8 -*-
import tweepy
import csv

# APIs from dev environment
##Four keys here: consumer key, consumer key secret, access key, access key secret
ckey = '34jYgpQ3kswuFlA4nYNKgfJjr' 
ckey_secret = 'FJTyCCzlcVScDocOCNc3omEAVXD4ZpMppiJ3ufamVmXqd2wjeD'
akey = '1034815350821593098-U0VnXpnIUtgFgcU1KmvZmpfnkYjPyL'
akey_secret = '3M7zM2PJCUcbAChK48r1NrmlAn7h03wZwIMjehHaww34N'

# for bold text
bold = "\033[1m"
reset = "\033[0;0m"

def get_tweets(user_name):
    
    # Use Tweepy package to do authentication
    aut = tweepy.OAuthHandler(ckey, ckey_secret)
    aut.set_access_token(akey, akey_secret)
    api = tweepy.API(aut)
    
    #define a tweet holder
    tweets_holder = []
    
    #Hold incoming new tweets, 500 tweets each time
    add_in_tweets = api.user_timeline(screen_name=user_name, count = 500)
    # print(add_in_tweets)
    
    # If the twitter account has no tweet, let the user try another account
    if add_in_tweets == []:
        print(bold + "Attention: This twitter user has no tweet in history, please try another one." + reset)
        return get_tweets(input("Input the username(e.g. @xxxxx) whose tweets you wanna see: "))
    
    #Put all tweets into the holder
    tweets_holder.extend(add_in_tweets)

    # Need further modification here!!
    
    oldest = tweets_holder[-1].id - 1

    # Keep getting tweets until there's nothing left

    while len(add_in_tweets) > 0:
        
        # We set the maximum equal to the oldest tweet to avoid duplication
        
        add_in_tweets = api.user_timeline(screen_name=user_name, count = 500, max_id = oldest)

        # Extend tweets holder to save the new tweets

        tweets_holder.extend(add_in_tweets)

        # Update the oldest tweet

        oldest = tweets_holder[-1].id - 1
        print ('Completed downloading %s number of tweets' % len(tweets_holder))
        

        # transforming all tweets into a csv

    output = [[tweet.id_str, tweet.created_at, tweet.text.encode('utf-8')] for tweet in tweets_holder]

    # writing to the csv

    with open(user_name + '_tweets_history.csv', 'w', encoding='utf8') as f:
        writer = csv.writer(f)
        writer.writerow(['ID', 'Create Time', 'Content'])
        writer.writerows(output)
            
if __name__ == '__main__':

    # Enter the username
    # The username is followed by '@'. You should include '@'. For example, President Donald Trump's
    # username is: @realDonaldTrump

    get_tweets(input("Input the username(e.g. @xxxxx) whose tweets you wanna see: "))
    

In [ ]:
#Visualizing the output

In [1]:
# load all necessary libraries
import numpy as np
import pandas as pd
import wordcloud
import matplotlib.pyplot as plt
from os import path
from PIL import Image

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3265, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-700da3a513a9>", line 3, in <module>
    import pandas as pd
ModuleNotFoundError: No module named 'pandas'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2016, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/site-packages/IPython/core

ModuleNotFoundError: No module named 'pandas'

In [3]:
# Generating word map(regular)

# text read-in
# address need modification
text = open(r'txt address').read()

# create a word cloud image
wordmap = WordCloud().generate(text)

# display the image
plt.imshow(wordcloud,interpolation="bilinear" )
plt.axis("off")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'txt address'

In [ ]:
# Generating flag-shaped worldmap

# Define mask
# image link??
flag_us_mask = np.array(Image.open("img/USflag.png"))
# Should add more flags


image_colors = ImageColorGenerator(flag_us_mask)
plt.figure(figsize=(10,8))
plt.imshow(flag_us.recolor(color_func=image_colors), interpolation="bilinear")
plt.axis("off")
plt.show()